<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">22.418 · Aprenentatge automàtic</p>
<p style="margin: 0; text-align:right;">Grau en Ciència de Dades Aplicada</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis de Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Content-based filtering

On this notebook, we will work with a movies dataset that can be downloaded from: https://www.kaggle.com/rounakbanik/the-movies-dataset (only movies_metadata.csv is required). <br> 

We will use this dataset to recommend movies which are similar according to the plot description.

If there are some problems to download the file, this will be also provided at the classroom. Once downloaded, you need to upload it on the session storage (Files button on the menu from the left side).

This notebook is based on the code from: https://medium.com/analytics-vidhya/content-based-recommender-systems-in-python-2b330e01eb80

In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


## Load the dataset

In [2]:
!wget -O movies_metadata.csv https://github.com/jiyeon1997/movie-recommender-python/blob/master/movies_metadata.csv?raw=true
!ls -lah movies_metadata.csv

--2021-01-15 09:14:36--  https://github.com/jiyeon1997/movie-recommender-python/blob/master/movies_metadata.csv?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jiyeon1997/movie-recommender-python/raw/master/movies_metadata.csv [following]
--2021-01-15 09:14:36--  https://github.com/jiyeon1997/movie-recommender-python/raw/master/movies_metadata.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jiyeon1997/movie-recommender-python/master/movies_metadata.csv [following]
--2021-01-15 09:14:36--  https://raw.githubusercontent.com/jiyeon1997/movie-recommender-python/master/movies_metadata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubuserconte

In [3]:
#putting movies data on 'movies' dataframe
movies = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Lets see which data the movies dataframe have (print the first 3 rows):

In [4]:
movies.head(3)  

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Lets see the plot of some movies:

In [5]:
movies["overview"][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [6]:
movies["overview"][1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

We have a dataset of around 45466 movies which is good enough to build a model that will recommend us movies based on the plot. It is going to be very interesting. <br>

As a first step, we will use TfidfVectorizer which will basically convert our ‘overview’ (a text column ) into numerical. All the data science models run on numerical values since computers can only understand 0s and 1s.<br>


TF-IDF basically is Term Frequency-Inverse Document frequency. The number of features it creates is equal to the total number of distinct words used in the overview column and the values are directly proportional to the number of times a particular word is used and inversely proportional to the number of documents (movies here) in which the word is used. It will penalize a word even though a word has a huge number for a movie but is common to many movies. The words which occur multiple times but are common to many movies are anyways not so helpful in differentiating different movies.

In [7]:
tfidf = TfidfVectorizer(stop_words="english")
movies["overview"] = movies["overview"].fillna("")

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
overview_matrix = tfidf.fit_transform(movies["overview"])

#Output the shape of tfidf_matrix
overview_matrix.shape

(45466, 75827)

Now, we have a ‘tfidf’ feature matrix for all the movies. Every movie has 75927 number of features (words ). Now, in order to find the similarity between the movies, we will use the cosine_similarity. In our case, the linear_kernel function will compute the same for us.

Cosine_Similarity is basically a measure of the similarity between 2 vectors. This measure is the cosine of the angle between them. Here, we have 75927 features (tfidf values) for each movie. Let us now find the similarity matrix using linear_kernel function:

In [ ]:
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

You will see that probably the previous code has made your session crashed after using all available RAM. This is due to the large size of movies (45,466). Notice that we are trying to compute a similarity matrix of size NxN where N is the number of movies.

In order to reduce the memory requirements we will work with a reduced dataset using the first 20,000 movies from the original dataset. Notice that after your session crash you will need to run again the previous cells (except for the one that causes the session crash).

In [ ]:
movies = movies[:20000]
tfidf = TfidfVectorizer(stop_words="english")
movies["overview"] = movies["overview"].fillna("")

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
overview_matrix = tfidf.fit_transform(movies["overview"])

#Output the shape of tfidf_matrix
overview_matrix.shape

In [ ]:
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

Now, let us create a series that maps the index of the matrix to movie names to make it easy for us to just feed in movie names and get the recommendation.

In [ ]:
#movies index mapping
mapping = pd.Series(movies.index,index = movies["title"])
mapping

Now, we will make a recommender function that will recommend us movies using cosine_similarity. Our function will take a movie name as input and then find the top K movies using the cosine similarity matrix we found above.

In [ ]:
def recommend_movies_based_on_plot(movie_input, num_movies):
    movie_index = mapping[movie_input]
    
    #get similarity values with other movies
    #similarity_score is the list of index and similarity matrix
    similarity_score = list(enumerate(similarity_matrix[movie_index]))
        
    #sort in descending order the similarity score of movie inputted with all the other movies
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        
    # Get the scores of the 15 most similar movies. Ignore the first movie.
    similarity_score = similarity_score[1:num_movies]
        
    #return movie names using the mapping series
    movie_indices = [i[0] for i in similarity_score]
    return (movies["title"].iloc[movie_indices])

Lets now try to get a recommendation for the movie ‘Star Wars’ from the above recommendation function and see what it outputs.

In [ ]:
recommend_movies_based_on_plot("Star Wars",10)

## Exercise

One of the parameters of the TfidfVectorizer is max_features. By default, a vocabulary composed by all the words from the plots of the movies is used (except for those words considered as common words such as articles, prepositions, etc. which form what is known as stop list). In our case, this vocabulary consists of 47,487 words. As a result, each movie is represented by a feature vector of length 47,487 and movies are compared by using these feature vectors.

However, the max_features parameter available in the TfidfVectorizer allows to set a maximum feature vector length by considering only the N most frequent words on this dataset, where N = max_features. As an exercise, we propose that you analyze how affects the use of this parameter on the recommender system. Try to use different values, analyze the computational time for computing the similarity matrix as well as how the movies recommended change depending on it.